---
Tom Curran

Problem Set \#4

MACS 30100

Monday February 5, 2018

---

Multivariable logistic (logit) regression (3 points). In this problem, you will use the Auto.csv dataset from Exercise 1. We will study the factors that make miles per gallon high or low. Create a binary variable mpg high that equals 1 if mpg high≥ median(mpg high) and equals 0 if mpg high< me- dian(mpg high).

a) Use statsmodel.api to estimate the logistic regression of mpg high on the regressors from Exercise 1: number of cylinders (cyl), engine displace- ment (dspl), horsepower (hpwr), vehicle weight (wgt), acceleration (accl), vehicle year (yr), vehicle origin (orgn). Make sure to include a constant term. Report all the regressors that have coefficients that are statistically significant at the 5% level (p¡0.05).

$$
Pr(mpg\_high = 1|X\beta) = \frac{e^{X \beta}}{1 + e^{X \beta}}
$$

$$
X \beta = \beta_0 + \beta_1 cyl_i + \beta_2 dspl_i + \beta_3 hpwr_i + \beta_4 wgt_i + \beta_5 accl_i + \beta_6 yr_i + \beta_7 orgn_i 
$$

In [204]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
df = pd.read_csv("Auto.csv", na_values = "?")
df2 = df
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 9 columns):
mpg             397 non-null float64
cylinders       397 non-null int64
displacement    397 non-null float64
horsepower      392 non-null float64
weight          397 non-null int64
acceleration    397 non-null float64
year            397 non-null int64
origin          397 non-null int64
name            397 non-null object
dtypes: float64(4), int64(4), object(1)
memory usage: 28.0+ KB
None


In [205]:
df2[df2.horsepower.isna()==True]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
32,25.0,4,98.0,NaN,2046,19.0,71,1,ford pinto
126,21.0,6,200.0,NaN,2875,17.0,74,1,ford maverick
330,40.9,4,85.0,NaN,1835,17.3,80,2,renault lecar deluxe
336,23.6,4,140.0,NaN,2905,14.3,80,1,ford mustang cobra
354,34.5,4,100.0,NaN,2320,15.8,81,2,renault 18i


In [206]:
df2['horsepower'] = df2.horsepower.fillna(df2.horsepower.mean())

df2[df2.horsepower.isna()==True]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name


In [207]:
df2.drop(['name'], axis = 1,inplace = True)

df2['constant'] = 1

df2['mpg_high'] = df2.mpg.apply(lambda x: 1 if x > np.median(df2.mpg) else 0)

df2.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,constant,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,1,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,1,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,1,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,1,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,1,0


In [209]:
logit_model = sm.Logit(df2['mpg_high'], df2[['cylinders','displacement', 'weight', 'acceleration','year', 'origin', 'constant','horsepower']], missing = 'drop')

logit_model_summary = logit_model.fit().summary()

print(logit_model_summary)

Optimization terminated successfully.
         Current function value: 0.189595
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:               mpg_high   No. Observations:                  397
Model:                          Logit   Df Residuals:                      389
Method:                           MLE   Df Model:                            7
Date:                Sun, 04 Feb 2018   Pseudo R-squ.:                  0.7262
Time:                        11:26:18   Log-Likelihood:                -75.269
converged:                       True   LL-Null:                       -274.90
                                        LLR p-value:                 3.449e-82
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
cylinders       -0.0644      0.434     -0.148      0.882      -0.916       0.787
displacement    -0.0015

In [210]:
x = df2.ix[:, (2,3,4,5,6,7,8,9)].values

y = df2.ix[:,-1].values

/Users/thomascurran/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [211]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = .5, random_state = 10)

In [212]:
logreg = LogisticRegression()

logreg.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [213]:
y_pred = logreg.predict(x_test)

confusion_matrix = metrics.confusion_matrix(y_test, y_pred)

print(confusion_matrix)

[[108   3]
 [  2  86]]


In [215]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.98      0.97      0.98       111
          1       0.97      0.98      0.97        88

avg / total       0.97      0.97      0.97       199

